In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
data=pd.read_csv('C:Desktop/data.csv',encoding='GBK')

In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
#y_train=data.status
#data.drop(['status'],axis=1,inplace=True)

In [4]:
#去掉重复的
data_cols=data.columns.values.tolist()
for col in data_cols:
    if data[col].value_counts().count()==1 :
        data.drop(col,axis=1,inplace=True)
data.drop(['id_name','trade_no','custid'],axis=1,inplace=True)
     
reg_preference_for_trad_mapping={
    '一线城市':1,
    '二线城市':2,
    '三线城市':3,
    '境外':4,
    '其他城市':5
}
data.reg_preference_for_trad=data.reg_preference_for_trad.map(reg_preference_for_trad_mapping)
data.reg_preference_for_trad.fillna(stats.mode(data.reg_preference_for_trad)[0][0],inplace=True)

data.latest_query_time=pd.to_datetime(data.latest_query_time)
data['latest_query_time_year']=data.latest_query_time.dt.year
data['latest_query_time_month']=data.latest_query_time.dt.month
data['latest_query_time_day']=data.latest_query_time.dt.day
data.loans_latest_time=pd.to_datetime(data.loans_latest_time)
data['loans_latest_time_year']=data.loans_latest_time.dt.year
data['loans_latest_time_month']=data.loans_latest_time.dt.month
data['loans_latest_time_day']=data.loans_latest_time.dt.day
data.drop(['latest_query_time','loans_latest_time'],axis=1,inplace=True)

In [5]:
data.student_feature.fillna('0',inplace=True)
data.dropna(thresh=80,inplace=True)
data.fillna(data.median(),inplace=True)

In [6]:
y_data=data.status
data.drop('status',axis=1,inplace=True)
x_train,x_test,y_train,y_test=train_test_split(data,y_data,test_size=0.3,random_state=2018)
x_test.head()

,Unnamed: 0,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,student_feature,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,first_transaction_day,trans_day_last_12_month,apply_score,apply_credibility,query_org_count,query_finance_count,query_cash_count,query_sum_count,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,latest_query_time_year,latest_query_time_month,latest_query_time_day,loans_latest_time_year,loans_latest_time_month,loans_latest_time_day
1131,2944,0.02,0.88,8500,1.76,1.00,0.741,28.0,9.0,8.0,3.0,1,31870,0,31.0,20150814.0,717700,486.0,0.25,60730,6.0,36950,0.05,1040,0.55,3.0,8.0,8.0,8.0,8.0,1.0,0.0,3.0,3.0,27.0,2000,18.0,0.0,280,27950,0.0,1011.0,82.0,638.0,79.0,8.0,5.0,3.0,10.0,2.0,5.0,5.0,603.0,76.0,71.0,71.0,0.0,20.0,5.0,15.0,0.0,1.0,26.0,101.0,2.0,0.0,0.0,338.0,2200.0,76.0,15.0,15.0,5000.0,1966.0,21700.0,79.0,5.0,5.0,49200.0,24000.0,17.0,68.0,2018.0,4.0,20.0,2018.0,2.0,28.0
3410,8586,0.09,0.91,400,1.14,0.52,0.242,16.0,85.0,69.0,4.0,0,5650,0,32.0,20130101.0,59450,126.0,0.70,690,6.0,4660,1.00,490,0.70,1.0,0.0,85.0,0.0,90.0,0.0,1.0,1.0,2.0,0.0,650,1.0,0.0,0,0,0.0,1966.0,83.0,531.0,75.0,4.0,3.0,1.0,6.0,2.0,4.0,6.0,486.0,77.0,8.0,6.0,1.0,5.0,2.0,3.0,0.0,1.0,4.0,6.0,3.0,0.0,0.0,298.0,700.0,78.0,3.0,3.0,1000.0,666.0,4400.0,77.0,2.0,3.0,5400.0,3600.0,12.0,71.0,2018.0,4.0,22.0,2018.0,2.0,22.0
366,898,0.00,0.54,0,1.03,0.86,0.297,14.0,23.0,27.0,3.0,0,31060,0,25.0,20151031.0,173520,145.0,0.55,25250,6.0,36870,0.65,2950,0.30,3.0,5.0,23.0,5.0,23.0,0.0,0.0,1.0,2.0,0.0,1050,2.0,0.0,1050,58780,0.0,933.0,95.0,531.0,83.0,2.0,1.0,1.0,3.0,1.0,2.0,2.0,495.0,75.0,13.0,13.0,1.0,4.0,0.0,4.0,0.0,0.0,3.0,14.0,0.0,0.0,0.0,353.0,3200.0,75.0,4.0,4.0,5400.0,2900.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,124.0,2018.0,4.0,10.0,2018.0,1.0,3.0
4084,10317,0.04,0.96,500,0.62,0.68,0.233,19.0,23.0,17.0,4.0,0,5310,0,39.0,20130602.0,58330,146.0,0.65,5850,7.0,1210,0.40,770,0.60,3.0,10.0,18.0,10.0,18.0,0.0,1.0,4.0,4.0,0.0,500,2.0,1.0,0,80,0.0,1814.0,82.0,609.0,74.0,4.0,2.0,1.0,5.0,0.0,0.0,1.0,584.0,83.0,8.0,7.0,0.0,4.0,0.0,4.0,0.0,0.0,1.0,8.0,1.0,0.0,0.0,334.0,1300.0,83.0,4.0,5.0,2000.0,1250.0,0.0,0.0,0.0,0.0,0.0,0.0,135.0,177.0,2017.0,12.0,23.0,2017.0,11.0,11.0
440,1077,0.02,0.86,20000,1.58,0.92,0.569,29.0,12.0,12.0,2.0,0,9790,0,33.0,20150328.0,375420,399.0,0.20,56240,7.0,19140,0.05,1330,0.50,1.0,6.0,10.0,6.0,10.0,3.0,1.0,5.0,5.0,35.0,4120,6.0,0.0,440,30890,0.0,1150.0,82.0,545.0,75.0,10.0,4.0,3.0,12.0,7.0,9

In [7]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [11]:
from sklearn.model_selection import GridSearchCV
RF=RandomForestClassifier()
parameters={'max_depth':range(1,10,3),'n_estimators':[100,200,300]}
forest = RandomForestClassifier(mini_samples_leaf=3,random_state=1,n_jobs=1)
reserchCV=GridSearchCV(forest,parameters,cv=3)
forest.fit(x_train,y_train)


AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'

In [15]:
forest = RandomForestClassifier(max_depth=7,n_estimators=300,min_samples_leaf=3,random_state=1,n_jobs=1)
forest.fit(x_train_scaled,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [27]:
from sklearn.feature_selection import SelectFromModel
importance=forest.feature_importances_
importance

In [33]:
importance.tolist()

[0.0,
 2.799863932493255e-05,
 0.0003215899541321043,
 0.0003395797899331078,
 0.0003752261724304975,
 0.0012827036584440422,
 0.0013841534315666528,
 0.0014789846290241063,
 0.0015722832265741282,
 0.0018828449835517859,
 0.0023124908923318668,
 0.0028364396161502064,
 0.0035313471696900287,
 0.0038029087202865043,
 0.0038979585289808916,
 0.004119713805001163,
 0.004180892223473923,
 0.004467364253719055,
 0.004510653244405755,
 0.004603856398905312,
 0.004683704038829709,
 0.004754371116288995,
 0.004938701929856092,
 0.005031313742786938,
 0.005145991090900059,
 0.00529318863253793,
 0.0056837766945243735,
 0.005712689287295449,
 0.005798117214765178,
 0.005908331340931259,
 0.005932927933637798,
 0.0059720024335673475,
 0.006193181403822276,
 0.006312882115214604,
 0.006480079872303023,
 0.006508791100766962,
 0.00660395924627532,
 0.006611684203904358,
 0.006738779648145083,
 0.006766259891902292,
 0.007001434568435048,
 0.007066341321841908,
 0.0071894324410006335,
 0.0072450446

In [26]:
data.head()

,Unnamed: 0,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,student_feature,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,first_transaction_day,trans_day_last_12_month,apply_score,apply_credibility,query_org_count,query_finance_count,query_cash_count,query_sum_count,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,latest_query_time_year,latest_query_time_month,latest_query_time_day,loans_latest_time_year,loans_latest_time_month,loans_latest_time_day
0,5,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,3.0,0,19890,0,30.0,20130817.0,149050,151.0,0.40,34030,7.0,3920,0.15,1020,0.55,1.0,4.0,19.0,4.0,19.0,1.0,1.0,2.0,2.0,5.0,2170,6.0,0.0,1970,18040,0.0,1738.0,85.0,583.0,79.0,8.0,2.0,6.0,10.0,2.0,5.0,8.0,552.0,73.0,37.0,34.0,2.0,10.0,1.0,9.0,1.0,1.0,13.0,37.0,7.0,1.0,0.0,341.0,2200.0,72.0,9.0,10.0,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0,2018.0,4.0,25.0,2018.0,4.0,19.0
1,10,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,4.0,1,16970,0,23.0,20160402.0,302910,224.0,0.35,10590,5.0,6950,0.05,1210,0.50,1.0,13.0,30.0,13.0,30.0,0.0,0.0,3.0,3.0,330.0,2100,9.0,0.0,1820,15680,0.0,779.0,84.0,653.0,73.0,7.0,4.0,2.0,8.0,2.0,6.0,8.0,635.0,76.0,37.0,36.0,0.0,17.0,5.0,12.0,2.0,2.0,8.0,49.0,4.0,2.0,1.0,353.0,2000.0,74.0,12.0,12.0,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0,2018.0,5.0,3.0,2018.0,5.0,5.0
2,12,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,1.0,0,9710,0,9.0,20170617.0,11520,31.0,1.00,5710,5.0,840,0.65,570,0.65,1.0,0.0,68.0,0.0,68.0,0.0,3.0,6.0,6.0,0.0,0,3.0,0.0,0,0,0.0,338.0,95.0,654.0,76.0,11.0,5.0,5.0,16.0,5.0,5.0,14.0,633.0,83.0,4.0,2.0,0.0,3.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,1.0,157.0,1500.0,77.0,2.0,2.0,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0,2018.0,5.0,5.0,2018.0,5.0,1.0
3,13,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,3.0,0,6210,0,33.0,20130516.0,491130,360.0,0.15,91690,7.0,46850,0.05,1290,0.45,3.0,6.0,8.0,6.0,8.0,0.0,1.0,8.0,8.0,31700.0,8140,9.0,0.0,2700,27970,0.0,1831.0,82.0,595.0,79.0,12.0,7.0,4.0,22.0,3.0,16.0,17.0,542.0,75.0,85.0,81.0,4.0,22.0,5.0,17.0,2.0,4.0,34.0,91.0,26.0,2.0,0.0,355.0,1800.0,74.0,17.0,18.0,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0,2018.0,5.0,5.0,2018.0,5.0,3.0
4,14,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,1.0,0,11150,0,12.0,20170312.0,61470,63.0,0.65,9770,6.0,760,1.00,1110,0.50,1.0,0.0,66.0,0.0,66.0,0.0,3.0,3.0,3.0,0.0,1000,3.0,0.0,0,6410,0.0,435.0,88.0,541.0,75.0,11.0,3.0,4.0,14.0,6.0,8.0,